In [ ]:
from conf import *
import pyspark
import pandas as pd
import subprocess
import time

sc.stop()
# Create new config
conf = pyspark.SparkConf().setAll([("spark.driver.maxResultSize", '16g'), ('spark.executor.memoryOverhead', '16g'), ('spark.executor.memory', '16g')])

SparkSession.builder.config(conf=conf)
spark = SparkSession.builder.appName('test').getOrCreate()
sc = spark.sparkContext
sc.addPyFile('file:/event.py')
sc.addPyFile('file:/interval_tree.py')
storage_level = pyspark.StorageLevel.MEMORY_AND_DISK
sc.setLogLevel("OFF")

start = time.time()

traffic_path = 'file:/datasets/TrafficEvents_Aug16_Dec20_Publish.csv'
weather_path = 'file:/datasets/WeatherEvents_Aug16_Dec20_Publish.csv'


def format_airport_row(r: str) -> str:
    parts = r.split(',')
    return (str(int(parts[0])),parts[1])

print("Loading Traffic and Weather events...")
input_rdd = sc.textFile(traffic_path)
input_rdd2 = sc.textFile(weather_path)
input_rdd3 = sc.textFile(airport_path).filter(lambda it: not it.startswith('Zip,')).map(format_airport_row)                                                                                                          
print("%d Traffic events\n%d Weather events" % (input_rdd.count(),input_rdd2.count()))

la_airport = input_rdd3.filter(lambda it: it[0] in zips_to_filter)
airport_to_filter = la_airport.map(lambda it: it[1]).distinct().collect()


print(f'\n{CITY} zipCodes:')
print(zips_to_filter)
print(f'\n{CITY} airportCodes:')
print(airport_to_filter)

traffic_rdd_map = input_rdd.filter(lambda it: it.split(',')[18] in zips_to_filter)
weather_rdd_map = input_rdd2.filter(lambda it: it.split(',')[8] in airport_to_filter)

traffic_rdd_map.saveAsTextFile('file:/datasets/subset_traffic')
weather_rdd_map.saveAsTextFile('file:/datasets/subset_weather')

subprocess.call(f'cat subset_traffic/part* > TrafficEvents_Aug16_Dec20_Publish_{CITY}.csv', shell=True, cwd='datasets/')
subprocess.call(f'cat subset_weather/part* > WeatherEvents_Aug16_Dec20_Publish_{CITY}.csv', shell=True, cwd='datasets/')
subprocess.call('rm -r subset_*', shell=True, cwd='datasets/')

print("%d Traffic events\n%d Weather events" % (traffic_rdd_map.count(),weather_rdd_map.count()))
print("Elapsed Time: %.2fs" % (time.time()-start))